In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, sum

In [ ]:
# Spark session & context
spark = SparkSession.builder.master('local[16]').getOrCreate()
sc = spark.sparkContext

# # Sum of the first 100 whole numbers
# rdd = sc.parallelize(range(100000 + 1))
# rdd.sum()

In [ ]:
# Read in the gun.csv file
df = spark.read.csv('gun_data.csv', header=True, inferSchema=True)

In [ ]:
print(df.head())

In [ ]:
# Print the columns
df.columns

In [ ]:
# remove any na values
df = df.na.drop()

In [ ]:
# Total rows of data
df.count()

In [ ]:
# transform n_killed into integer
from pyspark.sql.functions import col, sum
df = df.withColumn('n_killed', col('n_killed').cast('integer'))

In [ ]:
# sum of n_killed
df.groupBy().sum('n_killed').show()

In [ ]:
# Sort by number of times a state appears
df.groupBy('state').count().sort('count', ascending=False).show()

In [ ]:
# Sort by state and n_killed
df.groupBy('state').sum('n_killed').sort('sum(n_killed)', ascending=False).show()

In [ ]:
# make a new column for the year, extract from the date column
from pyspark.sql.functions import year
df = df.withColumn('year', year('date'))

In [ ]:
# sort by year frequency
df.groupBy('year').count().sort('count', ascending=False).show()

In [ ]:
# sort by year and n_killed
df.groupBy('year').sum('n_killed').sort('sum(n_killed)', ascending=False).show()

In [ ]:
# print df schema
print(df.printSchema())

In [ ]:
# Run k means clustering on longitude and latitude
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator

In [ ]:
# show a row of longitude
df.select('longitude').show()

In [ ]:
# Transform longitude and latitude into a float
df = df.withColumn('longitude', col('longitude').cast('float'))

In [ ]:
# Transform latitude into a float
df = df.withColumn('latitude', col('latitude').cast('float'))

In [ ]:
# Vector assembler
assembler = VectorAssembler(inputCols=['longitude', 'latitude'], outputCol='features')

In [ ]:
# Transform the data
df_kmeans = assembler.transform(df)

# Create the kmeans model
kmeans = KMeans().setK(2).setSeed(1)

In [ ]:
# Drop any rows with missing data
df_kmeans = df_kmeans.na.drop()

# Drop any 0 values
df_kmeans = df_kmeans.filter((df_kmeans.longitude != 0) & (df_kmeans.latitude != 0))

In [ ]:
# Fit the model
model = kmeans.fit(df_kmeans.sample(False, 0.1, seed=42))

In [ ]:
# Make predictions
predictions = model.transform(df_kmeans)


In [ ]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

# Show the silhouette score
silhouette = evaluator.evaluate(predictions)

In [ ]:
# Print the silhouette score
print("Silhouette with squared euclidean distance = " + str(silhouette))

In [ ]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [ ]:
# Show the predictions
predictions.show()